In [95]:
import numpy as np
import random
from random import randint
from gurobipy import *
import pandas as pd
#from random import seed

In [256]:
Manufacturing_plants = 2
Distribution = 3
Market = 4
Products = 2
Outsourced = 2
p = 0.5

In [183]:
# Scenario parameters
a_si = [[1,1], [1,0], [0,1]] # don't include [0,0]
b_sj = [[1,1,1], [1,0,1], [1,1,0], [1,0,0], [0,1,1], [0,1,0], [0,0,1]] # don't include [0,0,0]

In [184]:
Scenarios = [[x,y] for x in a_si for y in b_sj]

In [185]:
Scenarios

[[[1, 1], [1, 1, 1]],
 [[1, 1], [1, 0, 1]],
 [[1, 1], [1, 1, 0]],
 [[1, 1], [1, 0, 0]],
 [[1, 1], [0, 1, 1]],
 [[1, 1], [0, 1, 0]],
 [[1, 1], [0, 0, 1]],
 [[1, 0], [1, 1, 1]],
 [[1, 0], [1, 0, 1]],
 [[1, 0], [1, 1, 0]],
 [[1, 0], [1, 0, 0]],
 [[1, 0], [0, 1, 1]],
 [[1, 0], [0, 1, 0]],
 [[1, 0], [0, 0, 1]],
 [[0, 1], [1, 1, 1]],
 [[0, 1], [1, 0, 1]],
 [[0, 1], [1, 1, 0]],
 [[0, 1], [1, 0, 0]],
 [[0, 1], [0, 1, 1]],
 [[0, 1], [0, 1, 0]],
 [[0, 1], [0, 0, 1]]]

In [186]:
num_Scenarios = len(Scenarios)
p_scen = 1/num_Scenarios

In [187]:
# Product Demand
demand = np.random.randint(0,50,(num_Scenarios, Products,Market))

In [188]:
# Cost of opening
f_i = [200, 50]
f_j = [75, 100, 50]

In [189]:
# Unit cost of manufacturing product
Manufacturing_costs = np.random.uniform(0,2, (Manufacturing_plants,Products))

In [190]:
# Unit cost of transporting m from plant to DC
Transportation_i_j = np.random.uniform(0,2, (Products, Manufacturing_plants, Distribution))

In [191]:
# Unit cost of transporting m from DC to Market Zone
Transportation_j_k = np.random.uniform(0,2, (Products, Distribution, Market))

In [192]:
# Plant Capacities: Bigger capacities for the more expensive ones
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = np.random.randint(800,1000)
Capacities_i[1] = np.random.randint(200,400) 
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = np.random.randint(400, 600)
Capacities_j[1] = np.random.randint(600, 800)
Capacities_j[2] = np.random.randint(200,400)
Capacities_l = np.random.randint(50,100, (Products,Outsourced)) # in terms of products 

In [193]:
# Cost of purchasing product m from supplier l (assume only 1 product type from each outsourcer)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(10,15, (Products, Outsourced))
Supplier_cost[1] = np.random.randint(15,20, (Products, Outsourced))

In [194]:
Supplier_cost

array([[[12.46058147, 11.29122194],
        [12.29567878, 14.90016288]],

       [[19.        , 16.        ],
        [19.        , 17.        ]]])

In [195]:
# Cost of transporting product m from outsourced facility l to j
T_O_DC = np.random.uniform(2, 5, (Products, Outsourced, Distribution))

In [196]:
# Cost of shipping product m from outsourced facility l to k
T_O_MZ = np.random.uniform(5, 7,(Products, Outsourced, Market))

In [197]:
# Product volume 
volume = np.random.uniform(2,3,(Products))

In [198]:
# unit cost of lost sales 
lost_sales = np.random.randint(25,50,(Market,Products))

In [199]:
# Initialize model variables
x_i = {} # opening manufacturing plant
x_j = {} # opening DC
U_km = {} # quantity lost sales
V_lm = {} # quantity products purchased from outsourcing
Q_im = {} # quantity produced
Y_ijm = {} # shipping i -> j
Z_jkm = {} # shipping j -> k
T_ljm = {} # shipping l -> j
T_lkm = {} # shipping l -> k
y_lm = {} # indicator variable for step function 

# Dictionaries for analysis 
Cost_dict = {}
Summary_dict = {} 

## Deterministic Model

In [229]:
grbModel_det = Model('deterministic')

In [230]:
def SetGurobiModel(scen):
    
    for i in range(Manufacturing_plants):
        x_i[i] = grbModel_det.addVar(vtype = GRB.BINARY)
    
    for j in range(Distribution):
        x_j[j] = grbModel_det.addVar(vtype = GRB.BINARY)
    
    for k in range(Market):
        for m in range(Products):
            U_km[k,m] = grbModel_det.addVar(vtype = GRB.INTEGER)
        
    for m in range(Products):
        for l in range(Outsourced):
            V_lm[m,l] = grbModel_det.addVar(vtype = GRB.INTEGER)
    
    for m in range(Products):
        for i in range(Manufacturing_plants):
            Q_im[m,i] = grbModel_det.addVar(vtype = GRB.INTEGER)
            
    for m in range(Products):
        for i in range(Manufacturing_plants):
            for j in range(Distribution):
                Y_ijm[m,i,j] = grbModel_det.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for j in range(Distribution):
            for k in range(Market): 
                Z_jkm[m,j,k] = grbModel_det.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for l in range(Outsourced):
            for j in range(Distribution):
                T_ljm[m,l,j] = grbModel_det.addVar(vtype = GRB.INTEGER)      
    
    for m in range(Products):
        for l in range(Outsourced):
            for k in range(Market):
                T_lkm[m,l,k] = grbModel_det.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for l in range(Outsourced):
            y_lm[m,l] = grbModel_det.addVar(vtype = GRB.BINARY)
                
    SetGrb_Obj()
    ModelCons(scen)


In [202]:
def SolveModel():
    
    grbModel_det.params.OutputFlag = 0
    grbModel_det.optimize()
    
    # get variable values 
    v_val_x_i = grbModel_det.getAttr('x', x_i)
    v_val_x_j = grbModel_det.getAttr('x', x_j)
    v_val_U_km = grbModel_det.getAttr('x', U_km)
    v_val_V_lm = grbModel_det.getAttr('x', V_lm)
    v_val_Q_im = grbModel_det.getAttr('x', Q_im)
    v_val_Y_ijm = grbModel_det.getAttr('x', Y_ijm)
    v_val_Z_jkm = grbModel_det.getAttr('x', Z_jkm)
    v_val_T_ljm = grbModel_det.getAttr('x', T_ljm)
    v_val_T_lkm = grbModel_det.getAttr('x', T_lkm)
    v_val_y_lm = grbModel_det.getAttr('x', y_lm)
    
    obj = grbModel_det.getObjective()
        
    return


In [203]:
# Objective

def SetGrb_Obj():

    grb_expr = LinExpr()

    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x_i[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x_j[j]    
    
        
    # Shipment 
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0
    
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y_ijm[m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z_jkm[m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T_ljm[m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T_lkm[m,l,k]
                    
    total_shipment = ship_1 + ship_2 + ship_3 + ship_4

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += Manufacturing_costs[i][m]*Q_im[m,i]
                

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += Supplier_cost[0][m][l]*V_lm[m,l] + Supplier_cost[1][m][l]*y_lm[m,l]                

    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += lost_sales[k][m]*U_km[k,m]                

    grb_expr += OC_1 + OC_2 + total_shipment + pr_cost + b_cost + l_cost
    
    grbModel_det.setObjective(grb_expr, GRB.MINIMIZE)
    
    return 

In [204]:
# Model Constraints

def ModelCons(scen):
    
    # Network Flow

    grbModel_det.addConstrs(quicksum(Y_ijm[m,i,j] for j in range(Distribution)) == Q_im[m,i]
                         for i in range(Manufacturing_plants) for m in range(Products))

    grbModel_det.addConstrs(quicksum(Z_jkm[m,j,k] for k in range(Market)) ==
                        (quicksum(Y_ijm[m,i,j] for i in range(Manufacturing_plants)) +
                         quicksum(T_ljm[m,l,j] for l in range(Outsourced))) 
                        for j in range(Distribution) for m in range(Products))

    grbModel_det.addConstrs((quicksum(Z_jkm[m,j,k] for j in range(Distribution)) +
                         quicksum(T_lkm[m,l,k] for l in range(Outsourced))) == (demand[scen][m][k] - U_km[k,m])
                         for k in range(Market) for m in range(Products))  
        
                    
    # Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
    grbModel_det.addConstrs(V_lm[m,l] == quicksum(T_ljm[m,l,j] for j in range(Distribution)) + 
                        quicksum(T_lkm[m,l,k] for k in range(Market))  
                        for m in range(Products) for l in range(Outsourced))
    
    
    # Capacity Constraints
    grbModel_det.addConstrs(quicksum(volume[m]*Q_im[m,i] for m in range(Products)) <= Scenarios[scen][0][i]*Capacities_i[i]*x_i[i] 
                         for i in range(Manufacturing_plants))
    
    grbModel_det.addConstrs(quicksum(volume[m]*Y_ijm[m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
                        quicksum(volume[m]*T_ljm[m,l,j] for l in range(Outsourced) for m in range(Products)) <= 
                        Scenarios[scen][1][j]*Capacities_j[j]*x_j[j]  
                        for j in range(Distribution))
    
    grbModel_det.addConstrs((V_lm[m,l] <= (Capacities_l[m][l])) 
                        for l in range(Outsourced) for m in range(Products))
    
    
    # Indicator variable constraints for step function (5 is arbitrary)
    grbModel_det.addConstrs((V_lm[m,l] - (Capacities_l[m][l] - 25)*y_lm[m,l])
                         <= 25  for m in range(Products) for l in range(Outsourced))
    
    
    return

In [205]:
def run_Model(scen):    
        
    SetGurobiModel(scen)
    SolveModel()

In [206]:
scen_objs = []

In [207]:
for s in range(num_Scenarios):
    run_Model(s)
    scen_objs += [grbModel_det.objval]

In [231]:
scen_objs

[906.1313410845494,
 866.436980364818,
 941.4917941014711,
 1147.0640993066795,
 905.1649386179537,
 1059.640918673483,
 1483.4049157754182,
 925.5429943661953,
 895.493262198209,
 1201.086552661203,
 1561.2599296831202,
 869.2885642506578,
 789.6085151383979,
 1238.6476900479747,
 2740.567799904877,
 1558.7216544223174,
 1436.7613450141653,
 1612.7626637135677,
 2275.4570146101714,
 2277.6885898974215,
 1134.033399136964]

## Stochastic Model

In [282]:
# Initialize model variables
x_i = {} # opening manufacturing plant
x_j = {} # opening DC
U_km = {} # quantity lost sales
V_lm = {} # quantity products purchased from outsourcing
Q_im = {} # quantity produced
Y_ijm = {} # shipping i -> j
Z_jkm = {} # shipping j -> k
T_ljm = {} # shipping l -> j
T_lkm = {} # shipping l -> k
y_lm = {} # indicator variable for step function 

# Dictionaries for analysis 
Cost_dict = {}
Summary_dict = {} 

In [283]:
grbModel = Model('stochastic')

In [284]:
def SetGurobiModel():
    
    for i in range(Manufacturing_plants):
        x_i[i] = grbModel.addVar(vtype = GRB.BINARY)
    
    for j in range(Distribution):
        x_j[j] = grbModel.addVar(vtype = GRB.BINARY)
    
    for s in range(num_Scenarios):
        for k in range(Market):
            for m in range(Products):
                U_km[s,k,m] = grbModel.addVar(vtype = GRB.INTEGER)
        
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                V_lm[s,m,l] = grbModel.addVar(vtype = GRB.INTEGER)
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for i in range(Manufacturing_plants):
                Q_im[s,m,i] = grbModel.addVar(vtype = GRB.INTEGER)
            
    for s in range(num_Scenarios):  
        for m in range(Products):
            for i in range(Manufacturing_plants):
                for j in range(Distribution):
                    Y_ijm[s,m,i,j] = grbModel.addVar(vtype = GRB.INTEGER)
                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for j in range(Distribution):
                for k in range(Market): 
                    Z_jkm[s,m,j,k] = grbModel.addVar(vtype = GRB.INTEGER)                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                for j in range(Distribution):
                    T_ljm[s,m,l,j] = grbModel.addVar(vtype = GRB.INTEGER)      
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                for k in range(Market):
                    T_lkm[s,m,l,k] = grbModel.addVar(vtype = GRB.INTEGER)
                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                y_lm[s,m,l] = grbModel.addVar(vtype = GRB.BINARY)
                
    SetGrb_Obj()
    ModelCons()

In [285]:
def SolveModel():
    
    grbModel.params.OutputFlag = 1
    grbModel.optimize()
    
    # get variable values 
    v_val_x_i = grbModel.getAttr('x', x_i)
    v_val_x_j = grbModel.getAttr('x', x_j)
    v_val_U_km = grbModel.getAttr('x', U_km)
    v_val_V_lm = grbModel.getAttr('x', V_lm)
    v_val_Q_im = grbModel.getAttr('x', Q_im)
    v_val_Y_ijm = grbModel.getAttr('x', Y_ijm)
    v_val_Z_jkm = grbModel.getAttr('x', Z_jkm)
    v_val_T_ljm = grbModel.getAttr('x', T_ljm)
    v_val_T_lkm = grbModel.getAttr('x', T_lkm)
    v_val_y_lm = grbModel.getAttr('x', y_lm)
    
    obj = grbModel.getObjective()
    print("obj val: ", obj.getValue())   
    
    Summary_dict['ObjVal'] = grbModel.objval
    Summary_dict["OpenMPs"] = np.sum(v_val_x_i.values())
    Summary_dict["OpenDCs"] = np.sum(v_val_x_j.values())
    
    for s in range(num_Scenarios):        
        Summary_dict["Purchasing_" + str(s)] = sum([v_val_V_lm[(s,m,l)] for m in range(Products) for l in range(Outsourced)])    
        Summary_dict["Production_" + str(s)] = sum([v_val_Q_im[(s,m,i)] for m in range(Products) for i in range(Manufacturing_plants)])
        Summary_dict["LostSales_" + str(s)] = sum([v_val_U_km[(s,k,m)] for m in range(Products) for k in range(Market)])
        Summary_dict["OutsourceToDC_" + str(s)] = sum([v_val_T_ljm[(s,m,l,j)] for m in range(Products) for l in range(Outsourced) for j in range(Distribution)])
        Summary_dict["OutsourceToMarket_" + str(s)] = sum([v_val_T_lkm[(s,m,l,k)] for m in range(Products) for l in range(Outsourced) for k in range(Market)]) 
    
    Cost_dict["Opening"] =  get_opening_costs(v_val_x_i, v_val_x_j)
    
    for s in range(num_Scenarios):
        Cost_dict["InHouseShipping_" + str(s)] = get_shipping_costs(s,v_val_Y_ijm, v_val_Z_jkm, v_val_T_ljm, v_val_T_lkm)[0]
        Cost_dict["OutsourceShipping_" + str(s)] = get_shipping_costs(s,v_val_Y_ijm, v_val_Z_jkm, v_val_T_ljm, v_val_T_lkm)[1]
        Cost_dict["Production_" + str(s)] = get_production_cost(s,v_val_Q_im)
        Cost_dict["Purchasing_" + str(s)] = get_purchase_costs(s,v_val_V_lm, v_val_y_lm)
        Cost_dict["LostSales_" + str(s)] = get_lost_cost(s,v_val_U_km)    
    
    return

In [286]:
# Objective

def SetGrb_Obj():

    grb_expr = LinExpr()

    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x_i[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x_j[j]   
    
    # Shipment 
        
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y_ijm[0,m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z_jkm[0,m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T_ljm[0,m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T_lkm[0,m,l,k]
                    
    total_shipment = ship_1 + ship_2 + ship_3 + ship_4

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += Manufacturing_costs[i][m]*Q_im[0,m,i]                

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += Supplier_cost[0][m][l]*V_lm[0,m,l] + Supplier_cost[1][m][l]*y_lm[0,m,l]                

    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += lost_sales[k][m]*U_km[0,k,m]                

    grb_expr += OC_1 + OC_2 + total_shipment + pr_cost + b_cost + l_cost
    
    grbModel.setObjective(grb_expr, GRB.MINIMIZE)
    
    return 


In [287]:
# Model Constraints

def ModelCons():
    
    # Network Flow

    grbModel.addConstrs(quicksum(Y_ijm[s,m,i,j] for j in range(Distribution)) == Q_im[s,m,i]
                        for s in range(num_Scenarios) for i in range(Manufacturing_plants) for m in range(Products))

    grbModel.addConstrs(quicksum(Z_jkm[s,m,j,k] for k in range(Market)) ==
                        (quicksum(Y_ijm[s,m,i,j] for i in range(Manufacturing_plants)) +
                         quicksum(T_ljm[s,m,l,j] for l in range(Outsourced))) for s in range(num_Scenarios)
                        for j in range(Distribution) for m in range(Products))

    grbModel.addConstrs((quicksum(Z_jkm[s,m,j,k] for j in range(Distribution)) +
                         quicksum(T_lkm[s,m,l,k] for l in range(Outsourced))) == (demand[s][m][k] - U_km[s,k,m])
                        for s in range(num_Scenarios) for k in range(Market) for m in range(Products))  
        
                    
    # Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
    grbModel.addConstrs(V_lm[s,m,l] == quicksum(T_ljm[s,m,l,j] for j in range(Distribution)) + 
                        quicksum(T_lkm[s,m,l,k] for k in range(Market)) for s in range(num_Scenarios) 
                        for m in range(Products) for l in range(Outsourced))
    
    
    # Capacity Constraints
    grbModel.addConstrs(quicksum(volume[m]*Q_im[s,m,i] for m in range(Products)) <= Scenarios[s][0][i]*Capacities_i[i]*x_i[i] 
                        for s in range(num_Scenarios) for i in range(Manufacturing_plants))
    
    grbModel.addConstrs(quicksum(volume[m]*Y_ijm[s,m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
                        quicksum(volume[m]*T_ljm[s,m,l,j] for l in range(Outsourced) for m in range(Products)) <= 
                        Scenarios[s][1][j]*Capacities_j[j]*x_j[j] for s in range(num_Scenarios) for s in range(num_Scenarios)
                        for j in range(Distribution))
    
    grbModel.addConstrs((V_lm[s,m,l] <= (Capacities_l[m][l])) for s in range(num_Scenarios)
                        for l in range(Outsourced) for m in range(Products))
    
    
    # Indicator variable constraints for step function (25 is arbitrary)
    grbModel.addConstrs((V_lm[s,m,l] - (Capacities_l[m][l] - 25)*y_lm[s,m,l])
                         <= 25 for s in range(num_Scenarios) for m in range(Products) for l in range(Outsourced))
    
    # Resilience Metric
    
    grbModel.addConstrs(quicksum(f_i[i]*x_i[i] for i in range(Manufacturing_plants))
                        + quicksum(f_j[j]*x_j[j] for j in range(Distribution))
                        + quicksum(Transportation_i_j[m][i][j]*Y_ijm[s,m,i,j] for i in range(Manufacturing_plants) 
                                  for j in range(Distribution) for m in range(Products))
                        + quicksum(Transportation_j_k[m][j][k]*Z_jkm[s,m,j,k] for j in range(Distribution)
                                   for k in range(Market) for m in range(Products))
                        + quicksum(T_O_DC[m][l][j]*T_ljm[s,m,l,j] for l in range(Outsourced) for j in range(Distribution)
                                   for m in range(Products))
                        + quicksum(T_O_MZ[m][l][k]*T_lkm[s,m,l,k] for l in range(Outsourced) for k in range(Market)
                                   for m in range(Products))
                        + quicksum(Manufacturing_costs[i][m]*Q_im[s,m,i] for i in range(Manufacturing_plants) 
                                   for m in range(Products))
                        + quicksum(Supplier_cost[0][m][l]*V_lm[s,m,l] + Supplier_cost[1][m][l]*y_lm[s,m,l]
                                   for l in range(Outsourced) for m in range(Products))
                        + quicksum(lost_sales[k][m]*U_km[s,k,m] for k in range(Market) for m in range(Products)) 
                        
                        <= (1 + 0.5)*scen_objs[s] for s in range(num_Scenarios))                                 
                                                                    
    
    
    return   


In [288]:
def get_opening_costs(x1, x2):
    
    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x1[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x2[j]

    Opening = np.round(OC_1 + OC_2)
    
    return(Opening)
   
def get_shipping_costs(scen, Y, Z, T1, T2):
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0

    # Shipment
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y[scen, m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z[scen,m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T1[scen,m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T2[scen,m,l,k]
    
    in_house_shipping = np.round(ship_1 + ship_2)

    outsourced_shipping = np.round(ship_3 + ship_4)
    
    return(in_house_shipping, outsourced_shipping)

def get_production_cost(scen, Q):

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += Manufacturing_costs[i][m]*Q[scen,m,i]
            
    return(np.round(pr_cost))

def get_purchase_costs(scen, V, y):    

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += Supplier_cost[0][m][l]*V[scen,m,l] + Supplier_cost[1][m][l]*y[scen,m,l]
            
    return(np.round(b_cost))

def get_lost_cost(scen,U):
    
    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += lost_sales[k][m]*U[scen,k,m]
            
    return(np.round(l_cost))

In [289]:
def run_Model():    
        
    SetGurobiModel()
    SolveModel()

In [290]:
def dict_to_dataframe(Dict):
    return pd.DataFrame([list(Dict.values())], columns = list(Dict.keys()))

In [291]:
run_Model()

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 2016 rows, 1769 columns and 16597 nonzeros
Model fingerprint: 0x61721485
Variable types: 0 continuous, 1769 integer (89 binary)
Coefficient statistics:
  Matrix range     [3e-02, 8e+02]
  Objective range  [3e-02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 1509 rows and 569 columns
Presolve time: 0.01s
Presolved: 507 rows, 1200 columns, 3483 nonzeros
Variable types: 0 continuous, 1200 integer (98 binary)

Root relaxation: objective 8.883205e+02, 616 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  888.32055    0  100          -  888.32055      -     -    0s
H    0     0                    1000.4348037  888.32055  11.2%     -    0s
     0     0  998.02942

In [292]:
num_correct = 0
for s in range(num_Scenarios):
    if (Summary_dict['Purchasing_'+str(s)] + Summary_dict['Production_' + str(s) ] + Summary_dict['LostSales_' + str(s)] == 
          np.sum(demand[s])):
        num_correct += 1
num_correct == num_Scenarios

True

In [293]:
Purchasing = [Summary_dict['Purchasing_' + str(s)] for s in range(num_Scenarios)]
Production = [Summary_dict['Production_' + str(s)] for s in range(num_Scenarios)]
LostSales = [Summary_dict['LostSales_' + str(s)] for s in range(num_Scenarios)]
OutsourceToDC = [Summary_dict['OutsourceToDC_' + str(s)] for s in range(num_Scenarios)]
OutsourceToMarket = [Summary_dict['OutsourceToMarket_' + str(s)] for s in range(num_Scenarios)]

In [294]:
Unit_df = pd.DataFrame(list(zip(Purchasing, Production, LostSales, OutsourceToDC, OutsourceToMarket)), 
             columns = ["Purchasing", "Production", "LostSales", "OutsourceToDC", "OutsourceToMarket"])

In [295]:
Unit_df

,Purchasing,Production,LostSales,OutsourceToDC,OutsourceToMarket
0,0.0,210.0,0.0,0.0,0.0
1,0.0,186.0,0.0,0.0,0.0
2,1.0,195.0,8.0,0.0,1.0
3,26.0,173.0,0.0,0.0,26.0
4,7.0,158.0,0.0,0.0,7.0
5,23.0,193.0,0.0,0.0,23.0
6,76.0,98.0,0.0,0.0,76.0
7,14.0,193.0,0.0,0.0,14.0
8,0.0,176.0,11.0,0.0,0.0
9,25.0,235.0,3.0,0.0,25.0


In [296]:
Purchasing_cost = [Cost_dict['Purchasing_' + str(s)] for s in range(num_Scenarios)]
Production_cost = [Cost_dict['Production_' + str(s)] for s in range(num_Scenarios)]
LostSales_cost = [Cost_dict['LostSales_' + str(s)] for s in range(num_Scenarios)]
InHouseShipping = [Cost_dict['InHouseShipping_' + str(s)] for s in range(num_Scenarios)]
OutsourceShipping = [Cost_dict['OutsourceShipping_' + str(s)] for s in range(num_Scenarios)]

In [297]:
Cost_dict['Opening'] + Purchasing_cost[0] + Production_cost[0] + InHouseShipping[0] + OutsourceShipping[0] + LostSales_cost[0]

998.0

In [298]:
grbModel.objVal

998.0294217512065

In [300]:
Cost_df = pd.DataFrame(list(zip(Purchasing_cost, Production_cost, LostSales_cost, InHouseShipping, OutsourceShipping)), 
             columns = ["Purchasing", "Production", "LostSales", "InHouseShipping", "OutsourceShipping"])

In [301]:
Cost_df

,Purchasing,Production,LostSales,InHouseShipping,OutsourceShipping
0,0.0,302.0,0.0,221.0,0.0
1,0.0,264.0,0.0,326.0,0.0
2,12.0,257.0,328.0,318.0,6.0
3,402.0,334.0,0.0,358.0,137.0
4,79.0,214.0,0.0,454.0,40.0
5,340.0,370.0,0.0,274.0,131.0
6,957.0,120.0,0.0,248.0,415.0
7,158.0,310.0,0.0,360.0,85.0
8,0.0,272.0,322.0,274.0,0.0
9,282.0,382.0,81.0,452.0,127.0


In [250]:
Summary_dict["OpenMPs"]

2.0

In [251]:
Summary_dict["OpenDCs"]

3.0

In [304]:
scen_costs = [Cost_dict['Opening'] + Purchasing_cost[s] + Production_cost[s] + InHouseShipping[s] + OutsourceShipping[s] + LostSales_cost[s] for s in range(num_Scenarios)]

In [307]:
scen_costs

[998.0,
 1065.0,
 1396.0,
 1706.0,
 1262.0,
 1590.0,
 2215.0,
 1388.0,
 1343.0,
 1799.0,
 2332.0,
 1302.0,
 981.0,
 1857.0,
 4106.0,
 2334.0,
 2080.0,
 2411.0,
 3036.0,
 3414.0,
 1661.0]

In [303]:
scen_objs

[906.1313410845494,
 866.436980364818,
 941.4917941014711,
 1147.0640993066795,
 905.1649386179537,
 1059.640918673483,
 1483.4049157754182,
 925.5429943661953,
 895.493262198209,
 1201.086552661203,
 1561.2599296831202,
 869.2885642506578,
 789.6085151383979,
 1238.6476900479747,
 2740.567799904877,
 1558.7216544223174,
 1436.7613450141653,
 1612.7626637135677,
 2275.4570146101714,
 2277.6885898974215,
 1134.033399136964]

In [308]:
(np.array(scen_costs) - np.array(scen_objs))/np.array(scen_objs)

array([0.10138559, 0.22917191, 0.48275323, 0.48727521, 0.39422104,
       0.50050831, 0.49318637, 0.49966021, 0.49973211, 0.49781046,
       0.49366544, 0.49777652, 0.24238782, 0.49921565, 0.49822967,
       0.49738088, 0.44770042, 0.49495028, 0.33423747, 0.49888796,
       0.46468349])